# 创建item-索引以及三元组

In [1]:
# 导入所需的库
import os
import pandas as pd

# 定义两个文件夹的路径
class_key_path = "class_key" # 课程文件夹的路径
route_key_path = "route_key" # 学习路线文件夹的路径

# 定义一个空的字典，用于存储item和索引的映射
item_index_dict = {}

# 定义一个空的列表，用于存储三元组   [(h,r,t),(h,r,t)...]
triples = []

# 定义一个索引计数器，从0开始
index = 0

# 遍历课程文件夹中的每个文件
for file in os.listdir(class_key_path):
    # 获取文件的绝对路径
    file_path = os.path.join(class_key_path, file)
    # 读取文件的内容，去掉换行符
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip().split("\n")
    # 获取文件名，去掉后缀
    file_name = file[:-4]
    # 判断文件名是否已经在字典中，如果不在，就添加到字典中，并更新索引
    if file_name not in item_index_dict:
        item_index_dict[file_name] = index
        index += 1
    # 遍历文件内容中的每个知识点
    for item in content:
        # 判断知识点是否已经在字典中，如果不在，就添加到字典中，并更新索引
        if item not in item_index_dict:
            item_index_dict[item] = index
            index += 1
        # 构建三元组，文件名作为头节点，知识点作为尾节点，关系为0
        triple = (item_index_dict[file_name], 0, item_index_dict[item])
        # 将三元组添加到列表中
        triples.append(triple)

# 遍历学习路线文件夹中的每个文件，重复上述步骤
for file in os.listdir(route_key_path):
    file_path = os.path.join(route_key_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read().strip().split("\n")
    file_name = file[:-4]
    if file_name not in item_index_dict:
        item_index_dict[file_name] = index
        index += 1
    for item in content:
        if item not in item_index_dict:
            item_index_dict[item] = index
            index += 1
        triple = (item_index_dict[file_name], 0, item_index_dict[item])
        triples.append(triple)

# 将字典转换为pandas数据，并保存为txt文件，格式为item名字\t索引值
item_index_df = pd.DataFrame(item_index_dict.items(), columns=["item", "index"])
item_index_df.to_csv("./process/item_index.txt", sep="\t", index=False)

# 将三元组列表转换为Pandas数据，并保存为txt文件，格式为h\t r\t t
triples_df = pd.DataFrame(triples, columns=["h", "r", "t"])
triples_df.to_csv("./process/triples.txt", sep="\t", index=False)


In [2]:
import pandas as pd
# 读取已保存的item_index.txt文件，并构建item_index_dict字典

def load_item_index(file_path):
    item_index_df = pd.read_csv(file_path, sep="\t")
    item_index_dict = dict(zip(item_index_df["item"], item_index_df["index"]))
    return item_index_dict, item_index_df

# 定义文件路径
saved_item_index_path = "./process/item_index.txt"

# 尝试加载已保存的item_index.txt文件
try:
    item_index_dict, item_index_df = load_item_index(saved_item_index_path)
    print("Item index loaded successfully.")
except FileNotFoundError:
    print("Item index file not found. You may need to create it.")

# 然后可以使用get_item_or_index函数
def get_item_or_index(x):
    # 如果输入的是item名，就返回索引值
    if isinstance(x, str):
        return item_index_dict.get(x, None)
    # 如果输入的是索引值，就返回item名
    elif isinstance(x, int):
        return item_index_df.loc[item_index_df["index"] == x, "item"].values[0]
    # 否则，返回None
    else:
        return None


Item index loaded successfully.


In [5]:
print(get_item_or_index("软件工程课程设计"))
print(get_item_or_index(425))

425
软件工程课程设计


# 创建用户评分数据（对课程喜好与否）

## 预处理

In [6]:
import pandas as pd
# 读取CSV文件
# 注意csv文件这里原始的是GBK的
df = pd.read_csv('./process/grade_data_test.csv', encoding="GBK", index_col=0)

# 计算每门科目的平均分和标准差
mean = df.mean(axis=0)
std = df.std(axis=0)

# 对每个学生的每门科目的成绩进行标准化，得到标准分 （消除不同难度和分布的影响）
Z = (df - mean) / std

# 设置阈值，筛选出学生成绩比较好的科目，用1表示感兴趣，用0表示不感兴趣
threshold = 0.1
I = Z.applymap(lambda x: 1 if x >= threshold else 0)

print(I.head())

                高等数学A1(下)  计算机网络实验  概率统计A1  高等数学A1(上)  计算机网络  计算机组成原理  \
学号                                                                      
20171684310268          0        0       0          0      0        1   
20176807310177          0        1       0          1      1        1   
20177402320036          0        0       0          0      1        1   
20177402320040          0        0       0          0      1        1   
20177403320063          0        0       0          0      1        1   

                C++程序设计实验  计算机组成原理实验  计算机网络课程设计  离散数学  ...  数据结构实验  编译原理  \
学号                                                     ...                 
20171684310268          1          1          0     0  ...       0     0   
20176807310177          1          0          0     1  ...       1     0   
20177402320036          0          1          0     0  ...       0     0   
20177402320040          0          0          0     0  ...       0     0   
20177403320063          0       

In [99]:
I.to_csv('./process/grade_data_rate.csv')

In [7]:
df = pd.read_csv('./process/grade_data_rate.csv', index_col=0)
df

,高等数学A1(下),计算机网络实验,概率统计A1,高等数学A1(上),计算机网络,计算机组成原理,C++程序设计实验,计算机组成原理实验,计算机网络课程设计,离散数学,...,数据结构实验,编译原理,编译原理课程设计,算法分析与设计实验,算法分析与设计,编译原理实验,数据库课程设计,数据挖掘实验,数据挖掘,操作系统
学号,,,,,,,,,,,,,,,,,,,,,
20171684310268,0,0,0,0,0,1,1,1,0,0,...,0,0,1,1,0,0,1,1,1,1
20176807310177,0,1,0,1,1,1,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
20177402320036,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,1,1
20177402320040,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20177403320063,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20197001310168,0,1,0,1,0,1,0,1,1,1,...,1,1,1,0,1,1,1,1,1,1
20197201310042,0,1,1,0,1,0,1,1,1,1,...,0,1,1,1,0,1,1,1,0,1
20197203310002,1,1,1,0,1,1,0,1,1,1,...,0,1,1,0,1,1,1,1,1,0


## 正式生成用户评分矩阵

In [8]:
# 导入所需的库
import pandas as pd

# 读取csv文件，将学号作为索引
df = pd.read_csv("./process/grade_data_rate.csv", index_col=0)

# 定义一个空的列表，用于存储用户评分
ratings = []

# 定义一个用户索引计数器，从0开始
user_index = 0

# 遍历数据框的每一行
for row in df.iterrows():
    # 获取行的索引和值
    index, values = row
    # 遍历行的每一列
    for col, value in values.iteritems():
        # 获取列的名字，即科目名
        item = col
        # 获取科目的索引，使用前面定义的函数
        item_index = get_item_or_index(item)
        # 构建用户评分，格式为用户索引\t科目索引\t用户评分
        # 构建用户评分向量，格式为[用户索引，物品， 评分]
        rating = [user_index, item_index, value]
        # rating = f"{user_index}\t{item_index}\t{value}"
        # 将用户评分添加到列表中
        ratings.append(rating)
    # 更新用户索引
    user_index += 1

# 将列表转换为数据框，并保存为txt文件
ratings_df = pd.DataFrame(ratings, columns=["user_index", "item_index", "rating"])
ratings_df.to_csv("./process/ratings.txt", sep="\t", index=False, float_format="%.0f")

In [9]:
ratings_df

,user_index,item_index,rating
0,0,454,0
1,0,372,0
2,0,185,0
3,0,442,0
4,0,71,0
...,...,...,...
4859,127,334,0
4860,127,140,0
4861,127,165,0
4862,127,133,0


# 判断有没有缺失值的函数

In [10]:
import pandas as pd
import numpy as np

def find_missing_positions(df):
    # 判断是否存在缺失值
    missing_mask = df.isna()

    if missing_mask.any().any():
        # 获取缺失值的行列索引
        missing_positions = np.where(missing_mask)
        return missing_positions
    else:
        return None



# 查找缺失值的位置
result = find_missing_positions(df)

if result:
    print("DataFrame中存在缺失值，位置为:")
    for row, col in zip(*result):
        print(f"行: {row}, 列: {col}")
else:
    print("DataFrame中没有缺失值.")

DataFrame中没有缺失值.


In [13]:
df.columns[19]

'软件工程课程设计'